# EEG Signals-based Emotion Recognition
## Methodology
In recent weeks, I have explored more recent literature on this topic and encountered a newer [review paper](https://www.sciencedirect.com/science/article/pii/S0010482523009150?via%3Dihub). I observed that methods achieving the highest performance metrics (such as accuracy and precision) often incorporate either the spatial component of EEG signals or both spatial and temporal components. A common approach, following data acquisition and preprocessing, involves projecting the time-series EEG data onto brain images, then using model architectures that leverage both spatial and temporal features simultaneously. This approach inspired me to consider combining time-domain and frequency-domain information in the preprocessing stage by applying time-frequency domain analysis techniques.

## Why Time-Frequency Domain Analysis?
- **Non-Stationary Nature of EEG**: EEG signals often exhibit transient events and oscillatory behavior that standard time-domain or frequency-domain analyses alone may not capture.
- **Comprehensive Representation**: Time-frequency methods provide a two-dimensional view of the signal, revealing how its frequency content evolves over time.
- **Enhanced Feature Extraction**: This approach allows the deep learning model to learn features related to both temporal and spectral information, improving its performance in emotion recognition.

## Choosing the Morlet Wavelet Transform
### Characteristics of the Morlet Wavelet
- **Complex Structure**: The Morlet wavelet is a Gaussian-modulated sinusoidal wave, which provides a good balance between time and frequency localization.
- **High Frequency Resolution**: Ideal for analyzing signals with oscillatory behavior, such as brain waves, making it suitable for EEG analysis.
- **Good for Dynamic Changes**: The Morlet wavelet is effective for detecting and analyzing changes in frequency over time, which aligns with the nature of emotional responses in EEG data.

### Reasons for Choosing the Morlet Wavelet
1. **Oscillatory Component Detection**: Emotions are often associated with different brain wave patterns (e.g., alpha, beta), which are effectively captured by the Morlet wavelet's oscillatory structure.
2. **Time-Frequency Localization**: The Morlet wavelet provides a balance that allows for detailed analysis of how frequencies change over time, essential for mapping emotional states.
3. **Proven Utility in EEG Analysis**: The Morlet wavelet is widely used in the field for time-frequency analysis due to its reliability in preserving the dynamic characteristics of non-stationary signals like EEG.

## Conclusion
I encountered a [paper](https://www.sciencedirect.com/science/article/pii/S1746809421002457) that employs a similar preprocessing approach to the one proposed here. Notably, they also used the SEED dataset, which provides a valuable point of comparison for evaluating methods and performance metrics in emotion recognition with EEG data. I welcome any suggestions or recommendations for further reading, particularly on  preprocessing techniques of the signals.
## Implementation Note
Most of algorithms are already implemented in the scientific Python module **Scipy (`scipy.signal`)** which can be used. I have some programming experience, to make amendements so it fits specific use cases as well.


In [ ]:
!pip install PyWavelets

In [ ]:
import numpy as np # linear programming operations
import pandas as pd # data manipulation and preprocessing
import matplotlib.pyplot as plt # for plotting and data visualization
import torch # pytorch for Deep Learning
import torch.nn as nn # neural network module
import torch.nn.functional as F # functional module
import torch.optim as optim # model optimzer module
import pywt # CWT
from sklearn.decomposition import PCA #PCA
import os
from google.colab import drive # for mounting google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DEAP = [pd.read_pickle(f"/content/drive/MyDrive/DEAP/data_preprocessed_python/s{i}.dat") for i in range(1,33)]

In [ ]:
data = [DEAP[i]['data'] for i in range(0,32)]
# data: 32 Subjects x 40 Trials (63 secs each with first 3 secs removed) x 40 Data rows (including 32 EEG channels) x 8064 (Timeseries data downsampled to 128 Hz amd EOG-filtered with bandpass filter of (4-45 Hz) average to common reference)
labels = [DEAP[i]['labels'] for i in range(0,32)]
# lables: 32 Subjects x 40 Trials x 4 Ratings (Valence,Arousal,Dominance,Liking)

In [ ]:
processed_data = []

for subject in data:
    subject_processed = []
    for trial in subject:
        # Convert each trial (40x8064) to a Pandas DataFrame
        df_trial = pd.DataFrame(trial)

        # Drop the last 8 rows (We only need the 32 EEG channels)
        df_trial_dropped = df_trial.iloc[:-8]

        # Append the processed trial as a NumPy array
        subject_processed.append(df_trial_dropped.to_numpy())

    # Add processed subject to the list
    processed_data.append(subject_processed)

In [ ]:
processed_data_flattened = [trial for subject in processed_data for trial in subject]
labels_flattened = [trial for subject_labels in labels for trial in subject_labels]

In [ ]:
print(len(processed_data_flattened),"\n", processed_data_flattened[0].shape)


1280 
 (32, 8064)


In [ ]:
# Concatenate all trials to create a single DataFrame
eeg_signal_names = [
    "Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1",
    "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4",
    "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8",
    "PO4", "O2"
]
processed_data_df_list = [pd.DataFrame(trial) for trial in processed_data_flattened]
for i in range(len(processed_data_df_list)):
    processed_data_df_list[i] = processed_data_df_list[i].transpose()
# Convert labels_flattened to a DataFrame
labels_df = pd.DataFrame(labels_flattened, columns=['Valence', 'Arousal', 'Dominance', 'Liking'])

In [ ]:
len(processed_data_df_list)

1280

In [ ]:
processed_data_df_list[0].shape

(8064, 32)

In [ ]:
labels_df.shape

(1280, 4)

In [ ]:
pca = PCA(n_components=16)
for i in range(len(processed_data_df_list)):
  processed_data_df_list[i] = pca.fit_transform(processed_data_df_list[i])
data = np.array(processed_data_df_list)

In [ ]:
data.shape

(1280, 8064, 16)

In [ ]:
scales = np.arange(1, 128)  # freq range
sampling_frequency = 128  # 128 Hz for DEAP
output_dir = "scalograms" # Save for later use
os.makedirs(output_dir, exist_ok=True)

# Perform CWT and store scalograms
for trial in range(0,1281):
    trial_scalograms = []
    for channel in range(0,16): # 16 PCs
        signal = data[trial, :, channel] # Extract single trial-channel data
        coef, freqs = pywt.cwt(signal, scales, 'cmor', sampling_period=1/sampling_frequency) # CWT
        scalogram = np.abs(coef) ** 2
        # Save the scalogram as a .npy file
        filename = f"scalogram_trial{trial}_channel{channel}.png"
        filepath = os.path.join(output_dir, filename)
        # Save using a dedicated figure to ensure proper scaling
        plt.figure(figsize=(12, 6))  # Adjust figure size for better resolution
        plt.imshow(scalogram, extent=[0, 8064, freqs[-1], freqs[0]], aspect='auto', cmap='jet')
        plt.axis('off')  # No axes
        plt.tight_layout(pad=0)
        plt.savefig(filepath, bbox_inches='tight', pad_inches=0)
        plt.close()  # Close the figure to free memory
        print(f"Saved scalograms for Trial {trial + 1} and Channel {channel + 1}")

    # (Optional) Save trial-level scalograms if needed
    # trial_filename = f"scalograms_trial{trial}.npy"
    # trial_filepath = os.path.join(output_dir, trial_filename)
    # np.save(trial_filepath, trial_scalograms)


/usr/local/lib/python3.10/dist-packages/pywt/_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
